# Описание проекта
По представленным данным необходимо провести исследовательский анализ предсказания возраста моллюска `галиотиса` (он же *морское ушко*). Необходимо провести однофакторный анализ, построить матрицу корреляций и пр. Также необходимо в ходе выполнения проекта использовать разные библиотеки обработки данных для сравнения скорости их работы.

# Порядок и критерии успешной реализации проекта

В рамках проекта необходимо реализовать следующие пункты:
1.  [Общее исследование данных](#Общее-исследование-данных) (**1 балл**):
  * проверить размеры таблицы,
  * число столбцов,
  * количество пропусков в столбцах.
2.  [Однофакторный анализ](#Однофакторный-анализ) (**3 балла**):
  * по каждому столбцу посмотреть его распределение,
  * если есть пропуски и ошибочные значения - обработать их,
  * посмотреть связь с целевой переменной .
3.  [Построить матрицу корреляций](#Построение-матрицы-корреляций) (или ее аналог для категориальных признаков)  (**2 балла**):
  * использовать корреляции,
  * тест `χ2χ2`,
  * `ANOVA`.
4.  [Провести анализ целевой переменной](#Анализ-целевой-переменной) (**1 балл**):
  * на какое распределение больше всего похожа ее гистограмма?
  * какие проблемы в связи с этим могут возникнуть при обучении моделей?
5.  Увеличить размер данных простым дублированием до *1_000_000* строк в датасете. Провести [сравнение скорости работы](#Сравнение-библиотек) `Pandas` и `Polars` на увеличенном датасете по параметрам:
  * загрузка данных (**1 балл**)
  * фильтрация данных (**1 балл**)
  * агрегация данных (**1 балл**).


# Исходные данные

Предскажите возраст морского ушка по физическим измерениям

* `Sex` - пол моллюска - номинальный - `M`, `F` и `I` (*infant*)

* `Lenght` - измерение длины раковины в мм - непрерывная

* `Diameter` - диаметр (перпендикулярно длине) в мм - непрерывный

* `Height` - высота (с мясом в раковине) в мм - непрерывная

* `Whole weight` - вес целого моллюска в граммах - непрерывный

* `Shucked weight` - вес очищенного моллюска (мясо) в граммах - непрерывный

* `Viscera weight` - вес потрохов (после обескровливания) в граммах - непрерывный

* `Shell weight` - вес скорлупы (вес моллюска после высушивания) в граммах - непрерывный

* `Rings` - кольца (+1.5 дает возраст в годах) - целое число - целевой признак

# Выполнение проекта
## Общее исследование данных
Выполним общее исследование данных согласно плану

### Импорт библиотек
Вначале выполним импорт библиотек, используемых в рамках проекта

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.25.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


### Открытие датасета
Откроем датасет и просмотрим первые строки

In [2]:
abalone_url = 'https://raw.githubusercontent.com/aiedu-courses/stepik_eda_and_dev_tools/main/datasets/abalone.csv'
df = pd.read_csv(abalone_url)
df.head()

,Sex,Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight,Rings
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,M,0.440,0.365,0.125,NaN,0.2155,0.1140,0.155,10
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7


Уже по первым пяти строкам видно, что в датасете присутствуют пропуски. 

Подсчитаем, сколько в датасете строк и столбцов, а также какие типы данных у признаков.

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4177 entries, 0 to 4176
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Sex             4177 non-null   object 
 1   Length          4177 non-null   float64
 2   Diameter        4078 non-null   float64
 3   Height          4177 non-null   float64
 4   Whole weight    4078 non-null   float64
 5   Shucked weight  4177 non-null   float64
 6   Viscera weight  4177 non-null   float64
 7   Shell weight    4127 non-null   float64
 8   Rings           4177 non-null   int64  
dtypes: float64(7), int64(1), object(1)
memory usage: 293.8+ KB


In [4]:
df.shape

(4177, 9)

Как видно, датасет содержит немногим более 4К строк (достаточно немного) и 9 столбцов-признаков, как и обозначено в разделе [Исходные данные](#Исходные-данные). У всех признаков, кроме двух, тип данных float64, т.е. имеем типичный пример количественные данные. Признак `Sex` относится к качественным; содержит не только данные о поле моллюска, но и его _возрасте_. Признак `Rings` по описанию датасета является целевым и целочисленным. 

### Определение и обработка пропусков
Уже по первым строкам видно, что в датасете имеются пропуски. Выполним анализ всех признаков.

In [5]:
df.isnull().sum().sort_values(ascending=False)

Diameter          99
Whole weight      99
Shell weight      50
Sex                0
Length             0
Height             0
Shucked weight     0
Viscera weight     0
Rings              0
dtype: int64

На первый взгляд, пропусков немного, а двойное упоминание числа _99_ у двух признаков наводит на мысль, что они могут дублироваться в одних и тех же строках. Выведем случайные 10 строк для пропусков в признаке `Whole weight`. 

In [6]:
df[df['Whole weight'].isnull()].sample(10)

,Sex,Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight,Rings
2776,M,0.585,0.465,0.165,NaN,0.4025,0.1625,0.2740,10
1714,F,0.645,0.510,0.190,NaN,0.5730,0.3620,0.3600,10
1538,I,0.355,0.270,0.075,NaN,0.0790,0.0315,0.0540,6
3805,M,0.425,0.305,0.110,NaN,0.1730,0.0875,0.0975,9
1111,I,0.520,0.380,0.125,NaN,0.2880,0.1295,0.1670,8
1167,M,0.620,0.470,0.135,NaN,0.5315,0.2005,0.2475,8
561,I,0.515,0.400,0.120,NaN,0.2705,0.1790,0.1700,13
1810,M,0.645,0.500,0.160,NaN,0.6720,0.3260,0.3150,9
1256,I,0.425,0.310,0.090,NaN,0.1385,0.0650,0.0800,7
102,M,0.530,0.435,0.160,NaN,0.3160,0.1640,0.3350,15


Получаем результат, который нельзя однозначно охарактеризовать. Поэтому подсчитаем, сколько всего пропусков присутствует от общего количества строк. 

In [7]:
total_missing_rows = df[['Diameter', 'Whole weight', 'Shell weight']].isnull().any(axis=1).sum()
print(f'Пропуски содержатся в {round(100 * total_missing_rows / df.shape[0], 2)} процентах строк от общего числа')

Пропуски содержатся в 5.79 процентах строк от общего числа


В целом избавиться от чуть более 5 процентов строк было бы вполне оправдано, если их природа более ясна. Сейчас же можно поступить иначе. Для признака `Shell weight` удалим все строки, поскольку они составляют чуть более 1% от общего числа строк. А для признаков `Diameter` и `Whole weight` используем метод линейной интерполяции для более _плавного_ заполнения пропусков. 

In [15]:
df.dropna(subset=['Shell weight'], inplace=True)
df.interpolate(method='linear', inplace=True)
print(f'осталось {df.isnull().sum().sort_values(ascending=False).sum()} пропусков\nосталось {df.shape[0]} строк в датасете')

осталось 0 пропусков
осталось 4127 строк в датасете


### Определение и обработка уникальных значений
Хоть это и не требуется в рамках проекта напрямую, но для качественного признака (а он у нас единственный в датасете) можно рассмотреть, соответствует ли описанию данных имеющиеся значения.

In [16]:
df['Sex'].value_counts()

M    1432
I    1254
F    1248
f     193
Name: Sex, dtype: int64

Действительно в датасете для обозначения пола моллюска встречаются заглавное `F` и строчное описание `f`. Меж тем, в описании исходных данных фигурирует только верхний регистр. Возможно это и не скажется в дальнейшем, но, на всякий случай, приведём такие значения к единому верхнему регистру.

In [18]:
df['Sex'] = df['Sex'].str.upper()
df['Sex'].value_counts()

F    1441
M    1432
I    1254
Name: Sex, dtype: int64

## Однофакторный анализ

## Построение матрицы корреляций

## Анализ целевой переменной

## Сравнение библиотек

# Выводы